In [1]:
%matplotlib inline

# Preprocessing and Spike Sorting Tutorial

# Chapter 1: Importing Recording Data and Metadata

In [2]:
import os
import warnings
import glob
import pickle
import _pickle as cPickle
import imp
import git

/tmp/ipykernel_449297/3524477660.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"

In [4]:
from collections import defaultdict
import time
import json

In [61]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
import scipy.signal
from labellines import labelLine, labelLines
import numpy as np
from scipy.stats import mannwhitneyu

In [6]:
from spectral_connectivity import Multitaper, Connectivity

In [7]:
from probeinterface import get_probe
from probeinterface.plotting import plot_probe, plot_probe_group
from probeinterface import write_prb, read_prb

In [8]:
# Changing the figure size
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

The spikeinterface module by itself import only the spikeinterface.core submodule
which is not useful for end user



In [9]:
import spikeinterface

We need to import one by one different submodules separately (preferred).
There are 5 modules:

- :code:`extractors` : file IO
- :code:`toolkit` : processing toolkit for pre-, post-processing, validation, and automatic curation
- :code:`sorters` : Python wrappers of spike sorters
- :code:`comparison` : comparison of spike sorting output
- :code:`widgets` : visualization



In [10]:
import spikeinterface as si  # import core only
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw

In [11]:
import spikeinterface.core

We can also import all submodules at once with this
  this internally import core+extractors+toolkit+sorters+comparison+widgets+exporters

This is useful for notebooks but this is a more heavy import because internally many more dependency
are imported (scipy/sklearn/networkx/matplotlib/h5py...)



In [12]:
import spikeinterface.full as si

In [13]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (10,6)

# Part 1: Importing Data

## Loading in the Preprocessed LFP 

- Getting the root directory of the Github Repo to base the files off of

In [14]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [15]:
git_root

'/nancy/projects/reward_competition_extention'

- Getting a list of all the electrophysiological recording files
    - **NOTE**: If your recording file does not end with `.rec` or is in a different directory than `./data` then you must change `glob.glob({./path/to/recording_file.rec})` below. Where you replace `{./path/to/recording_file.rec}` with the path to your recording file without the brackets.

In [16]:
lfp_preprocessing_filepath_glob = glob.glob("./proc/*/lfp_preprocessing")

In [17]:
lfp_preprocessing_filepath_glob

['./proc/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged/lfp_preprocessing',
 './proc/20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged/lfp_preprocessing']

## Getting MetaData

In [18]:
subj_1_1_brain_region_to_channel = {"mPFC": "20", "MD": "28", "LH": "29", "BLA": "30", "vHPC": "31"}
subj_1_4_brain_region_to_channel = {"mPFC": "2", "MD": "28", "LH": "29", "BLA": "30", "vHPC": "31"}

In [19]:
tone_df = pd.read_csv("../2023_08_01_tone_timestamp/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.tone_times.csv")

In [20]:
tone_df

,Unnamed: 0,tone_timestamp,tone_index,rewarded_or_not,competition_winners,condition
0,0,1934930,1099250,rewarded,1.4,1.4
1,1,4334936,3499256,rewarded,1.1,1.1
2,2,6634931,5799251,rewarded,1.1,1.1
3,3,8434930,7599250,rewarded,1.4,1.4
4,4,9534930,8699250,rewarded,1.1,1.1
5,5,10534929,9699249,rewarded,1.4,1.4
6,6,12434926,11599246,rewarded,1.4,1.4
7,7,13934925,13099245,rewarded,1.4,1.4
8,8,16034924,15199244,rewarded,1.1,1.1
9,9,17034926,16199246,rewarded,1.1,1.1


In [21]:
tone_df["lfp_index"] = tone_df["tone_index"] // 20

## Getting LFP Traces

In [22]:
subj_1_1_lfp = spikeinterface.core.load_extractor(file_or_folder_or_dict="./proc/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged/lfp_preprocessing")


In [23]:
subj_1_4_lfp = spikeinterface.core.load_extractor(file_or_folder_or_dict="./proc/20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged/lfp_preprocessing")


In [24]:
lfp_preprocessing_filepath_glob

['./proc/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged/lfp_preprocessing',
 './proc/20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged/lfp_preprocessing']

In [25]:
subj_1_1_raw = se.read_spikegadgets("/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec", stream_id="trodes")
subj_1_4_raw = se.read_spikegadgets("/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec/20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged.rec", stream_id="trodes")

In [26]:
subj_1_1_raw

SpikeGadgetsRecordingExtractor: 32 channels - 1 segments - 20.0kHz - 3280.151s
  file_path: /scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec

In [27]:
subj_1_1_lfp = spikeinterface.preprocessing.bandpass_filter(subj_1_1_raw, freq_min=0.5, freq_max=300)
subj_1_4_lfp = spikeinterface.preprocessing.bandpass_filter(subj_1_4_raw, freq_min=0.5, freq_max=300)

In [28]:
subj_1_1_lfp = spikeinterface.preprocessing.notch_filter(subj_1_1_lfp, freq=60)
subj_1_4_lfp = spikeinterface.preprocessing.notch_filter(subj_1_4_lfp, freq=60)


In [29]:
resampled_frequency = 1000

In [30]:
subj_1_1_lfp = spikeinterface.preprocessing.resample(subj_1_1_lfp, resample_rate=1000)
subj_1_4_lfp = spikeinterface.preprocessing.resample(subj_1_4_lfp, resample_rate=1000)


In [31]:
subj_1_1_channels = subj_1_1_lfp.get_channel_ids()[[20, 28, 29, 30, 31]]

In [32]:
subj_1_4_channels = subj_1_4_lfp.get_channel_ids()[[2, 28, 29, 30, 31]]

# Power Calculation

In [33]:
tone_df["subj_1_1_condition"] = tone_df["condition"].apply(lambda x: "win" if x == "1.1" else ("lose" if x == "1.4" else x))
tone_df["subj_1_4_condition"] = tone_df["condition"].apply(lambda x: "win" if x == "1.4" else ("lose" if x == "1.1" else x))

In [34]:
tone_df

,Unnamed: 0,tone_timestamp,tone_index,rewarded_or_not,competition_winners,condition,lfp_index,subj_1_1_condition,subj_1_4_condition
0,0,1934930,1099250,rewarded,1.4,1.4,54962,lose,win
1,1,4334936,3499256,rewarded,1.1,1.1,174962,win,lose
2,2,6634931,5799251,rewarded,1.1,1.1,289962,win,lose
3,3,8434930,7599250,rewarded,1.4,1.4,379962,lose,win
4,4,9534930,8699250,rewarded,1.1,1.1,434962,win,lose
5,5,10534929,9699249,rewarded,1.4,1.4,484962,lose,win
6,6,12434926,11599246,rewarded,1.4,1.4,579962,lose,win
7,7,13934925,13099245,rewarded,1.4,1.4,654962,lose,win
8,8,16034924,15199244,rewarded,1.1,1.1,759962,win,lose
9,9,17034926,16199246,rewarded,1.1,1.1,809962,win,lose


In [35]:
tone_df.to_csv("./tone_df.csv")

In [36]:
def nested_dict():
    return defaultdict(nested_dict)

In [37]:
time_halfbandwidth_product = 1
time_window_duration = 1
time_window_step = 0.1

In [38]:
condition_to_brain_region = nested_dict()

In [39]:
for index, row in tone_df.iterrows():
    for brain_region, channel_id in subj_1_1_brain_region_to_channel.items():
        current_trace = subj_1_1_lfp.get_traces(channel_ids=[channel_id], start_frame=row["lfp_index"], end_frame=row["lfp_index"] + 1000 * 10)
        condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["traces"] = current_trace
        
                # Compute multitaper spectral estimate
        m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=current_trace,
                       sampling_frequency=resampled_frequency)
        
        # Sets up computing connectivity measures/power from multitaper spectral estimate
        c = Connectivity.from_multitaper(m)
        
        if condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_frequencies"]:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_frequencies"].append(c.frequencies)
        else:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_frequencies"] = [c.frequencies]

        if condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_power"]:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_power"].append(c.power().squeeze())
        else:
            condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["c_power"] = [c.power().squeeze()]


In [40]:
for index, row in tone_df.iterrows():
    for brain_region, channel_id in subj_1_4_brain_region_to_channel.items():
        current_trace = subj_1_4_lfp.get_traces(channel_ids=[channel_id], start_frame=row["lfp_index"], end_frame=row["lfp_index"] + 1000 * 10)
        condition_to_brain_region[row["subj_1_4_condition"]][brain_region]["traces"] = current_trace
        
                # Compute multitaper spectral estimate
        m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=current_trace,
                       sampling_frequency=resampled_frequency)
        
        # Sets up computing connectivity measures/power from multitaper spectral estimate
        c = Connectivity.from_multitaper(m)
        
        if condition_to_brain_region[row["subj_1_4_condition"]][brain_region]["c_frequencies"]:
            condition_to_brain_region[row["subj_1_4_condition"]][brain_region]["c_frequencies"].append(c.frequencies)
        else:
            condition_to_brain_region[row["subj_1_4_condition"]][brain_region]["c_frequencies"] = [c.frequencies]

        if condition_to_brain_region[row["subj_1_4_condition"]][brain_region]["c_power"]:
            condition_to_brain_region[row["subj_1_4_condition"]][brain_region]["c_power"].append(c.power().squeeze())
        else:
            condition_to_brain_region[row["subj_1_4_condition"]][brain_region]["c_power"] = [c.power().squeeze()]


In [41]:
len(condition_to_brain_region["win"]["mPFC"]["c_power"])

20

In [42]:
len(condition_to_brain_region["lose"]["mPFC"]["c_power"])

20

In [43]:
c.frequencies

array([0.000e+00, 1.000e-01, 2.000e-01, ..., 4.997e+02, 4.998e+02,
       4.999e+02])

In [44]:
losing_omission_freq_index = np.where((6 <= c.frequencies) & (12 >= c.frequencies))[0]

In [45]:
losing_omission_freq_index

array([ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120])

In [46]:
winning_rewarded_freq_index = np.where((2 <= c.frequencies) & (6 >= c.frequencies))[0]

In [47]:
winning_rewarded_freq_index

array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60])

In [71]:
freq_index_6_to_12 = np.where((6 <= c.frequencies) & (12 >= c.frequencies))[0]

In [72]:
freq_index_6_to_12

array([ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120])

In [73]:
freq_index_2_to_6 = np.where((2 <= c.frequencies) & (6 >= c.frequencies))[0]

In [74]:
freq_index_2_to_6

array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60])

In [75]:
freq_to_index = {"freq_6_to_12": freq_index_6_to_12, "freq_2_to_6": freq_index_2_to_6}

In [77]:
for freq, index in freq_to_index.items():
    print(freq)
    for condition, all_regions in condition_to_brain_region.items():   
        print(condition)
        for brain_region, value in all_regions.items():
            print(brain_region)
            current_all_power = [powers[index] for powers in value["c_power"]]
            per_trial_average_power = [np.mean(powers) for powers in current_all_power]
            condition_to_brain_region[condition][brain_region][freq]["per_trial_average_power"] = per_trial_average_power


freq_6_to_12
lose
mPFC
MD
LH
BLA
vHPC
win
mPFC
MD
LH
BLA
vHPC
rewarded
mPFC
MD
LH
BLA
vHPC
omission
mPFC
MD
LH
BLA
vHPC
freq_2_to_6
lose
mPFC
MD
LH
BLA
vHPC
win
mPFC
MD
LH
BLA
vHPC
rewarded
mPFC
MD
LH
BLA
vHPC
omission
mPFC
MD
LH
BLA
vHPC


In [78]:
condition

'omission'

In [79]:
condition_to_brain_region["omission"][brain_region][freq]["per_trial_average_power"]

[19106.397662734766,
 10316.44227739941,
 0.0,
 770.6277024348975,
 1307.076003479175,
 412.19792728639055]

In [55]:
condition_to_brain_region["lose"]["BLA"]["per_trial_average_power"]

[17607.89435654146,
 16276.936113462909,
 15529.707367988467,
 14537.348452993027,
 14463.026294339561,
 12032.833989303275,
 2402.858324795996,
 3108.4712870597614,
 2746.310518644133,
 2950.7382129881853,
 1870.1530132639266,
 2444.5241938727736,
 2356.716735716545,
 2701.406541792814,
 2053.2296837905587,
 2041.5210753963026,
 1685.4166079414704,
 2238.8221497280297,
 1972.816938144843,
 2114.798784909745]

In [56]:
condition_to_brain_region["lose"]["vHPC"]["per_trial_average_power"]

[11143.55033263804,
 11795.783796087177,
 14414.161257057964,
 15338.558762259041,
 10959.24701242391,
 14968.435154846575,
 2460.3028273038026,
 3302.4982341792165,
 2760.692288159805,
 3036.9459031018223,
 1963.262030749597,
 2449.450652629968,
 2315.368994713001,
 2694.267654444791,
 2535.7993266782387,
 2423.878643381053,
 1871.331765096666,
 2234.3863199200723,
 1876.177815457206,
 2170.0322911287994]

In [57]:
condition_to_brain_region["omission"]["BLA"]["per_trial_average_power"]

[8271.104701446875,
 10380.257637153258,
 0.0,
 1273.9730519215625,
 857.1197509515016,
 760.5300664577353]

In [58]:
condition_to_brain_region["omission"]["vHPC"]["per_trial_average_power"]

[8021.264767597676,
 8090.1145253560335,
 0.0,
 628.1843569372364,
 399.33895860243797,
 329.2391778328558]

In [62]:
mannwhitneyu(condition_to_brain_region["omission"]["vHPC"]["per_trial_average_power"], condition_to_brain_region["lose"]["vHPC"]["per_trial_average_power"])


MannwhitneyuResult(statistic=28.0, pvalue=0.05358120140728836)

In [63]:
mannwhitneyu(condition_to_brain_region["omission"]["BLA"]["per_trial_average_power"], condition_to_brain_region["lose"]["BLA"]["per_trial_average_power"])


MannwhitneyuResult(statistic=28.0, pvalue=0.05358120140728836)

In [64]:
len(per_trial_average_power)

6

In [108]:
brain_region_and_condition_to_pvalue = nested_dict()
for freq, index in freq_to_index.items():
    for brain_region in subj_1_1_brain_region_to_channel.keys():
        print(brain_region)
    
        u_statistic, p_value = mannwhitneyu(condition_to_brain_region["lose"][brain_region][freq]["per_trial_average_power"], condition_to_brain_region["omission"][brain_region][freq]["per_trial_average_power"])
        brain_region_and_condition_to_pvalue[brain_region]["losing_vs_omission"]["u_statistic"][freq] = u_statistic
        brain_region_and_condition_to_pvalue[brain_region]["losing_vs_omission"]["p_value"][freq] = p_value
    
        
        u_statistic, p_value = mannwhitneyu(condition_to_brain_region["win"][brain_region][freq]["per_trial_average_power"], condition_to_brain_region["rewarded"][brain_region][freq]["per_trial_average_power"])
        brain_region_and_condition_to_pvalue[brain_region]["winning_vs_rewarded"]["u_statistic"][freq] = u_statistic
        brain_region_and_condition_to_pvalue[brain_region]["winning_vs_rewarded"]["p_value"][freq] = p_value

mPFC
MD
LH
BLA
vHPC
mPFC
MD
LH
BLA
vHPC


In [109]:
brain_region_and_condition_to_pvalue

defaultdict(<function __main__.nested_dict()>,
            {'mPFC': defaultdict(<function __main__.nested_dict()>,
                         {'losing_vs_omission': defaultdict(<function __main__.nested_dict()>,
                                      {'u_statistic': defaultdict(<function __main__.nested_dict()>,
                                                   {'freq_6_to_12': 71.0,
                                                    'freq_2_to_6': 45.0}),
                                       'p_value': defaultdict(<function __main__.nested_dict()>,
                                                   {'freq_6_to_12': 0.5327368283890023,
                                                    'freq_2_to_6': 0.3874560222386309})}),
                          'winning_vs_rewarded': defaultdict(<function __main__.nested_dict()>,
                                      {'u_statistic': defaultdict(<function __main__.nested_dict()>,
                                                   {'freq_6_to_12':

In [124]:
# Convert the triple-nested dictionary to a DataFrame
lfp_wilconxon_df = pd.concat({(i, j): pd.DataFrame(k) for i, jk in brain_region_and_condition_to_pvalue.items() for j, k in jk.items()}, axis=0)

# Reset the index and rename the columns
lfp_wilconxon_df = lfp_wilconxon_df.reset_index()

In [125]:
lfp_wilconxon_df = lfp_wilconxon_df.rename(columns={"level_0": "brain_region", "level_1": "condition", "level_2": "frequency"})

In [130]:
lfp_wilconxon_df = lfp_wilconxon_df.sort_values(["brain_region", "condition", "frequency"]).reset_index(drop=True)[["brain_region", "condition", "frequency", "p_value", "u_statistic"]]

In [131]:
lfp_wilconxon_df

,brain_region,condition,frequency,p_value,u_statistic
0,BLA,losing_vs_omission,freq_2_to_6,0.218529,81.0
1,BLA,losing_vs_omission,freq_6_to_12,0.053581,92.0
2,BLA,winning_vs_rewarded,freq_2_to_6,0.115410,380.0
3,BLA,winning_vs_rewarded,freq_6_to_12,0.006471,438.0
4,LH,losing_vs_omission,freq_2_to_6,0.744473,54.0
5,LH,losing_vs_omission,freq_6_to_12,0.108214,87.0
6,LH,winning_vs_rewarded,freq_2_to_6,0.336832,349.0
7,LH,winning_vs_rewarded,freq_6_to_12,0.014459,424.0
8,MD,losing_vs_omission,freq_2_to_6,0.457030,47.0
9,MD,losing_vs_omission,freq_6_to_12,0.138957,85.0


In [132]:
lfp_wilconxon_df.to_csv("./lfp_wilcoxon.csv")

# Comparing Spikes

In [ ]:
raise ValueError()

In [ ]:
subj_1_4_lfp.get_traces(channel_ids=start_frame=100, end_frame=200)

In [ ]:
sorted(list(subj_1_1_brain_region_to_channel.values()))

In [ ]:
sorted(list(subj_1_4_brain_region_to_channel.values()))

In [ ]:
subj_1_1_lfp

In [ ]:
subj_1_4_lfp

In [ ]:
subj_1_1_traces = subj_1_1_lfp.get_traces(channel_ids=subj_1_1_lfp.get_channel_ids()[[20, 28, 29, 30, 31]])

In [ ]:
subj_1_4_traces = subj_1_4_lfp.get_traces(channel_ids=subj_1_4_lfp.get_channel_ids()[[2, 28, 29, 30, 31]])

In [ ]:
subj_1_4_traces

In [ ]:
subj_1_1_traces.shape

In [ ]:
# Compute multitaper spectral estimate
subj_1_1_m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=subj_1_1_traces,
               sampling_frequency=resampled_frequency)

# Sets up computing connectivity measures/power from multitaper spectral estimate
c = Connectivity.from_multitaper(m)

In [ ]:
### Reading in the recording
trodes_recording = se.read_spikegadgets(recording_path, stream_id="trodes")

### Preprocessing the recording
preprocessing_step_dict = defaultdict(dict)
preprocessing_step_dict["recording_raw"] = trodes_recording

recording_intermediate = copy.copy(trodes_recording)
# Bandpass Filtering, keeping all the points that are within a certain frequency range
recording_bandpass = spikeinterface.preprocessing.bandpass_filter(recording_intermediate, freq_min=0.5, freq_max=300)
recording_intermediate = spikeinterface.preprocessing.bandpass_filter(recording_intermediate, freq_min=0.5, freq_max=300)
preprocessing_step_dict["recording_bandpass"] = recording_bandpass
print(recording_bandpass)

# Notch Filtering, keeping all the points that are within a certain frequency range
recording_notch = spikeinterface.preprocessing.notch_filter(recording_intermediate, freq=60)
recording_intermediate = spikeinterface.preprocessing.notch_filter(recording_intermediate, freq=60)
preprocessing_step_dict["recording_notch"] = recording_notch
print(recording_bandpass)

# We are not going to run the resampling step because it causes issues with saving to file
# Resampling
recording_resample = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency)
preprocessing_step_dict["recording_resample"] = recording_resample
print(recording_resample)

In [ ]:
raise ValueError()

```
MD EIB: 0(1 on Nancy notebook) Spike Gadgets/Spike Interface: 0
vHPC EIB: 16(17 on Nancy notebook) Spike Gadgets/Spike Interface: 31
BLA EIB: 17(18 on Nancy notebook) Spike Gadgets/Spike Interface: 30
LH EIB: 18(19 on Nancy notebook) Spike Gadgets/Spike Interface: 29
```

In [ ]:
brain_region_to_channel = {"mPFC": "21", "MD": "16", "vHPC": "15", "BLA": "14", "LH": "13"}

In [ ]:
brain_region_to_channel = {k: v for k, v in sorted(brain_region_to_channel.items(), key=lambda item: item[1])}

In [ ]:
brain_region_to_channel.keys()

In [ ]:
brain_region_to_channel.values()

In [ ]:
channel_ids = brain_region_to_channel.values()
time_range=(1000, 1005)
resampled_frequency = 1000

In [ ]:
time_halfbandwidth_product=10

### Brain Region to Channel

```
- All Trodes
    - 21, 31, 15, 14, 13
- mPFC:
    - 21 (0 to 31 on Spike interface)
    - 22(1 to 32 on Trodes)
- MD
    - 1 (EIB)
        - Maybe actually 16
    - 31(0 to 31 Spike interface)
        - Maybe actually 16
    - 32(1 to 32 Trodes)
- vHPC
    - 17(EIB)
    - 15(0 to 31 on Spike interface)
    - 16(1 to 32 Trodes)
- BLA
    - 18(BLA)
    - 14(0 to 31 Spike interface)
    - 15(1 to 32 Trodes)
- LH 18
    - 19(EIB)
    - 13(0 to 31 Spike interface)
    - 14(1 to 32 Trodes)
```

In [ ]:
resampled_frequency = 1000

In [ ]:
recording_intermediate.channel_ids

In [ ]:
for preprocessed_lfp_path in lfp_preprocessing_filepath_glob:
    print(preprocessed_lfp_path)
    recording_intermediate = spikeinterface.core.load_extractor(file_or_folder_or_dict=preprocessed_lfp_path)
    recording_basename = os.path.basename(os.path.dirname(preprocessed_lfp_path))
    parent_output_directory = os.path.dirname(preprocessed_lfp_path)

    
    # Resampling
    recording_intermediate = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency) 
#     sw.plot_timeseries(recording_intermediate, time_range=time_range, channel_ids=channel_ids, show_channel_ids=True)

    # Getting the traces
    
    traces = recording_intermediate.get_traces(start_frame=1100*1000, end_frame=1160*1000)
    
    # Compute multitaper spectral estimate
    m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=traces,
                   sampling_frequency=resampled_frequency)
    
    # Sets up computing connectivity measures/power from multitaper spectral estimate
    c = Connectivity.from_multitaper(m)
    
    for num in range(8):
        channels = [int(num) for num in recording_intermediate.channel_ids[num*4: num*4+4]]
        
    
    
        fig, ax = plt.subplots()
        ax.set_xlabel("Frequency")
        ax.set_ylabel("Power")
        ax.plot(c.frequencies, c.power().squeeze()[:,channels], label=channels)
        ax.set_xlim(1,12)
        ax.legend(ncol=4)
        labelLines(ax.get_lines(), zorder=2.5)
        plt.title(recording_basename)
        os.makedirs('./proc/{}'.format(recording_basename), exist_ok=True)
        plt.savefig('./proc/{}/{}_ch_{}.png'.format(recording_basename, recording_basename, "-".join([str(ch) for ch in channels])))
        plt.show()


In [ ]:
for preprocessed_lfp_path in lfp_preprocessing_filepath_glob:
    print(preprocessed_lfp_path)
    recording_intermediate = spikeinterface.core.load_extractor(file_or_folder_or_dict=preprocessed_lfp_path)
    recording_basename = os.path.basename(os.path.dirname(preprocessed_lfp_path))
    parent_output_directory = os.path.dirname(preprocessed_lfp_path)

    
    # Resampling
    recording_intermediate = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency) 
#     sw.plot_timeseries(recording_intermediate, time_range=time_range, channel_ids=channel_ids, show_channel_ids=True)

    # Getting the traces
    
    traces = recording_intermediate.get_traces(start_frame=1000*1000, end_frame=1010*1000)
    
    # Compute multitaper spectral estimate
    m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=traces,
                   sampling_frequency=resampled_frequency)
    
    # Sets up computing connectivity measures/power from multitaper spectral estimate
    c = Connectivity.from_multitaper(m)
    
    for num in range(8):
        channels = [int(num) for num in recording_intermediate.channel_ids[num*4: num*4+4]]
        
    
    
        fig, ax = plt.subplots()
        ax.set_xlabel("Frequency")
        ax.set_ylabel("Power")
        ax.plot(c.frequencies, c.power().squeeze()[:,channels], label=channels)
        ax.set_xlim(1,12)
        ax.set_ylim(1,2000)

        ax.legend(ncol=4)
        labelLines(ax.get_lines(), zorder=2.5)
        plt.title(recording_basename)
        plt.show()


### Get all the traces

In [ ]:
raise ValueError()

In [ ]:
recording_root_to_everything = defaultdict(dict)

In [ ]:
resampled_frequency = 1000

In [ ]:
np_issue_files = []
for preprocessed_lfp_path in lfp_preprocessing_filepath_glob:
    try:
        print(preprocessed_lfp_path)
        recording_intermediate = spikeinterface.core.load_extractor(file_or_folder_or_dict=preprocessed_lfp_path)
        recording_basename = os.path.basename(os.path.dirname(preprocessed_lfp_path))
        parent_output_directory = os.path.dirname(preprocessed_lfp_path)

        # Resampling
        recording_intermediate = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency) 

        # Getting the traces
        
        traces = recording_intermediate.get_traces(start_frame=0, end_frame=recording_intermediate.get_num_samples()-1)
        # Saving the trace to a Numpy file
        traces_filename = "{}.traces.npy".format(recording_basename)
        np.save(file=os.path.join(parent_output_directory, traces_filename), arr=traces)
    except:
        np_issue_files.append(preprocessed_lfp_path)